## Fine-tuning XLS-R for Multi-Lingual ASR with Transformers

In [ ]:
!nvidia-smi

Mon Nov 13 16:03:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    22W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 63.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import tensorflow as tf

try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs or multi-GPU machines
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [ ]:
!pip install -q --upgrade ipython==7.34.0
!pip install -q --upgrade ipykernel==5.5.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.3 MB/s eta 0:00:00


1. Installing the necessary packages

In [ ]:
!pip install datasets
!pip install huggingface_hub
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install --upgrade git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.1 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ih5o2j93
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ih5o2j93
  Resolved https://github.com/huggingface/transformers to commit 3b5962131093ceab09f0540cb99d84c18c45035f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 20.2 M

### `torchaudio` is used to load audio files and `jiwer` to evaluate our fine-tuned model using the word error rate (WER)

In [ ]:
!pip install peft loralib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 10.6 MB/s eta 0:00:00


In [ ]:
!pip install accelerate

## Logging in to my huggingface account to be able to download dataset

In [ ]:
from huggingface_hub import login
login()

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
!pip install datasets --upgrade

## Loading Mozilla Common Voice 13.0 Swahili dataset

In [ ]:
from datasets import load_dataset

cv_swahili_train = load_dataset("mozilla-foundation/common_voice_13_0", "sw", split="train", use_auth_token=True)
cv_swahili_validate = load_dataset("mozilla-foundation/common_voice_13_0", "sw", split="validation", use_auth_token=True)
cv_swahili_test = load_dataset("mozilla-foundation/common_voice_13_0", "sw", split="test", use_auth_token=True)


Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 14928it [00:00, 149268.04it/s]
Reading metadata...: 32277it [00:00, 146876.61it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

...


Reading metadata...: 10978it [00:00, 155492.53it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 16471it [00:00, 164692.49it/s]
Reading metadata...: 34015it [00:00, 171004.78it/s]
Reading metadata...: 51116it [00:00, 163198.90it/s]
Reading metadata...: 67477it [00:00, 152986.59it/s]
Reading metadata...: 82868it [00:00, 148371.24it/s]
Reading metadata...: 97764it [00:00, 147809.16it/s]
Reading metadata...: 112581it [00:00, 147154.60it/s]
Reading metadata...: 127319it [00:00, 144890.92it/s]
Reading metadata...: 141823it [00:00, 141073.23it/s]
Reading metadata...: 155950it [00:01, 136750.47it/s]
Reading metadata...: 169651it [00:01, 131166.72it/s]
Reading metadata...: 182809it [00:01, 128135.32it/s]
Reading metadata...: 195742it [00:01, 128471.95it/s]
Reading metadata...: 209384it [00:01, 130762.48it/s]
Reading metadata...: 222612it [00:01, 131196.58it/s]
Reading metadata...: 235751it [00:01, 126716.70it/s]
Reading metadata...: 249907it [00:01, 130998.55it/s]
Reading metadata...: 264254it [00:01, 134636.48it/s]
Reading met

Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 14288it [00:00, 142870.26it/s]
Reading metadata...: 28576it [00:00, 142374.30it/s]
Reading metadata...: 60533it [00:00, 152267.00it/s]


## Function to show random elements from the train, validate, and test dataset

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(cv_swahili_train)

,client_id,path,audio,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,cf21aea1345888479b2c7ed1cfe8246dbe4f7f01acca6309b485ef697a43ab5d40bf26384e1d74f7a96b8b8f48f8d41785acdda4f828327b5858cb826ae7c367,/root/.cache/huggingface/datasets/downloads/extracted/e1af3253b811f6ea12d16411bc452950702d93aec19f951229c67078f572fb63/sw_train_0/common_voice_sw_34747868.mp3,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/e1af3253b811f6ea12d16411bc452950702d93aec19f951229c67078f572fb63/sw_train_0/common_voice_sw_34747868.mp3', 'array': [-1.7053025658242404e-13, -7.105427357601002e-13, 1.1368683772161603e-13, 1.9610979506978765e-12, 2.4726887204451486e-12, 2.1884716261411086e-12, 1.7621459846850485e-12, 9.094947017729282e-13, 2.5579538487363607e-13, 2.8421709430404007e-13, 1.9895196601282805e-13, 8.810729923425242e-13, 2.1032064978498966e-12, 2.5579538487363607e-12, 1.3642420526593924e-12, 1.7621459846850485e-12, 4.263256414560601e-12, 6.764366844436154e-12, 8.924416761146858e-12, 1.1212364370294381e-11, 1.1652900866465643e-11, 9.322320693172514e-12, 6.0254023992456496e-12, 3.069544618483633e-12, 1.1368683772161603e-13, -3.410605131648481e-12, -8.981260180007666e-12, -1.5347723092418164e-11, -2.0463630789890885e-11, -2.276578925375361e-11, -2.3163693185779266e-11, -1.9895196601282805e-11, -1.2732925824820995e-11, -6.707523425575346e-12, -5.9685589803848416e-12, -9.968914582714206e-12, -1.2150280781497713e-11, -7.972289495228324e-12, -1.5916157281026244e-12, -2.5011104298755527e-12, -1.4438228390645236e-11, -3.0254909688665066e-11, -4.260414243617561e-11, -5.03632691106759e-11, -5.4797055781818926e-11, -4.615685611497611e-11, -1.9667822925839573e-11, 1.1596057447604835e-11, 2.546585164964199e-11, 1.546140993013978e-11, -6.991740519879386e-12, -2.560796019679401e-11, -3.700506567838602e-11, -5.2864379540551454e-11, -7.867129170335829e-11, -9.981704351957887e-11, -1.0436451702844352e-10, -1.070930011337623e-10, -1.1937117960769683e-10, -1.291482476517558e-10, -1.2244072422618046e-10, -1.155626705440227e-10, -1.3034195944783278e-10, -1.6156320725713158e-10, -1.7456613932154141e-10, -1.397779669787269e-10, -7.071321306284517e-11, -1.8701484805205837e-11, -2.177102942368947e-11, -7.008793545537628e-11, -1.3153567124390975e-10, -1.9065282685915008e-10, -2.3990764930204023e-10, -2.601154847070575e-10, -2.3214852262753993e-10, -1.5711520973127335e-10, -7.048583938740194e-11, -4.1040948417503387e-11, -1.0948042472591624e-10, -2.3226220946526155e-10, -3.118998392892536e-10, -2.9672264645341784e-10, -2.1964297047816217e-10, -1.6189005691558123e-10, -1.7121237760875374e-10, -2.119691089319531e-10, -2.2293988877208903e-10, -2.2282620193436742e-10, -2.680735633475706e-10, -3.461764208623208e-10, -3.9028691389830783e-10, -3.7471181713044643e-10, -3.2514435588382185e-10, -2.5710278350743465e-10, -1.5489831639570184e-10, -2.4272139853565022e-11, 5.7980287238024175e-11, -5.7411853049416095e-12, -2.1589130483334884e-10, -4.5270098780747503e-10, ...], 'sampling_rate': 48000}",naam na katika hatua nyingine nchi ya ufaransa imesema inamtambua,2,1,,,,sw,
1,e1fe690dd9bd8e757c3f22114e14d0c0b0dda7a2a557ddf0176c60b856c3d75859c22556d39c0d9d458ce005fdeb5ec8670c147370dac6e1ba9f40a0f08faa11,/root/.cache/huggingface/datasets/downloads/extracted/e1af3253b811f6ea12d16411bc452950702d93aec19f951229c67078f572fb63/sw_train_0/common_voice_sw_30381124.mp3,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/e1af3253b811f6ea12d16411bc452950702d93aec19f951229c67078f572fb63/sw_train_0/common_voice_sw_30381124.mp3', 'array': [1.7053025658242404e-13, -4.888534022029489e-12, -5.8548721426632255e-12, -1.7053025658242404e-12, 2.8421709430404007e-13, 0.0, 1.6484591469634324e-12, 5.343281372915953e-12, 6.707523425575346e-12, 5.229594535194337e-12, 3.865352482534945e-12, 1.1368683772161603e-12, 9.094947017729282e-13, -2.1600499167107046e-12, -6.59383658785373e-12, -5.4569682106375694e-12, 1.9326762412674725e-12, 1.1596057447604835e-11, 2.5124791136477143

In [ ]:
columns_to_remove = ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"]
cv_swahili_train = cv_swahili_train.remove_columns(columns_to_remove)
cv_swahili_validate = cv_swahili_validate.remove_columns(columns_to_remove)
cv_swahili_test = cv_swahili_test.remove_columns(columns_to_remove)

## Regular expression to remove symbols since it does not map to any phoneme in the Swahili language

In [ ]:
import re

# Modify the chars_to_remove_regex pattern to include the additional symbols
chars_to_remove_regex = r'[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\(\)\*\=\_`\[\]\/\*°ː’•…]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [ ]:
cv_swahili_train = cv_swahili_train.map(remove_special_characters)
cv_swahili_validate = cv_swahili_validate.map(remove_special_characters)
cv_swahili_test = cv_swahili_test.map(remove_special_characters)

Map:   0%|          | 0/32277 [00:00<?, ? examples/s]

Map:   0%|          | 0/10977 [00:00<?, ? examples/s]

Map:   0%|          | 0/10978 [00:00<?, ? examples/s]

In [ ]:
def replace_extra_characters(batch, column_name="sentence"):
    character_replacements = {
        'µ': 'u',
        'á': 'a',
        'â': 'a',
        'ã': 'a',
        'å': 'a',
        'é': 'e',
        'è': 'e',
        'ë': 'e',
        'í': 'i',
        'ï': 'i',
        'ñ': 'n',
        'ó': 'o',
        'ö': 'o',
        'ø': 'o',
        'ú': 'u',
        'š': 's',
        'ū': 'u',
        'μ': 'u',
        'ụ': 'u'
    }

    for original, replacement in character_replacements.items():
        batch[column_name] = re.sub(re.escape(original), replacement, batch[column_name])

    # Remove multiple dots and tabs
    batch[column_name] = re.sub(r'\.\.\.+', '', batch[column_name])
    batch[column_name] = re.sub(r'\t', '', batch[column_name])

    return batch

In [ ]:
cv_swahili_train = cv_swahili_train.map(replace_extra_characters)
cv_swahili_validate = cv_swahili_validate.map(replace_extra_characters)
cv_swahili_test = cv_swahili_test.map(replace_extra_characters)

Map:   0%|          | 0/32277 [00:00<?, ? examples/s]

Map:   0%|          | 0/10977 [00:00<?, ? examples/s]

Map:   0%|          | 0/10978 [00:00<?, ? examples/s]

## Extracting the Swahili phoneme from the dataset transcript

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = cv_swahili_train.map(extract_all_chars, batched=True, batch_size=-1, remove_columns=cv_swahili_train.column_names)
vocab_validate = cv_swahili_validate.map(extract_all_chars, batched=True, batch_size=-1, remove_columns=cv_swahili_validate.column_names)
vocab_test = cv_swahili_test.map(extract_all_chars, batched=True, batch_size=-1, remove_columns=cv_swahili_test.column_names)

Map:   0%|          | 0/32277 [00:00<?, ? examples/s]

Map:   0%|          | 0/10977 [00:00<?, ? examples/s]

Map:   0%|          | 0/10978 [00:00<?, ? examples/s]

In [ ]:
# Convert "vocab" column from each dataset to sets and union them
vocab_set_train = set(vocab_train["vocab"][0])
vocab_set_validate = set(vocab_validate["vocab"][0])
vocab_set_test = set(vocab_test["vocab"][0])

# Merge vocabularies
vocab_set = vocab_set_train | vocab_set_validate | vocab_set_test

# Convert the result back to a list
vocab_list = list(vocab_set)

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict.pop('י', None)
vocab_dict

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
print(vocab_dict)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '|': 0}


In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

29

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from huggingface_hub import create_repo
create_repo("finetuning-wav2vec-large-swahili-asr-model_v10")

RepoUrl('https://huggingface.co/Joshua-Abok/finetuning-wav2vec-large-swahili-asr-model_v10', endpoint='https://huggingface.co', repo_type='model', repo_id='Joshua-Abok/finetuning-wav2vec-large-swahili-asr-model_v10')

In [ ]:
repo_name = "finetuning-wav2vec-large-swahili-asr-model_v10"
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/Joshua-Abok/finetuning-wav2vec-large-swahili-asr-model_v10/commit/dd3dbf71b15476cc11bf17e487cdb6b8626a9b6d', commit_message='Upload tokenizer', commit_description='', oid='dd3dbf71b15476cc11bf17e487cdb6b8626a9b6d', pr_url=None, pr_revision=None, pr_num=None)

## Create Wav2VecFeatureExtractor

Speech is a continuous signal, and, to be treated by computers, it first has to be discretized, which is usually called sampling. The sampling rate hereby plays an important role since it defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the real speech signal but also necessitates more values per second.

A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on. The same speech signals sampled at two different rates have a very different distribution. For example, doubling the sampling rate results in data points being twice as long. Thus, before fine-tuning a pretrained checkpoint of an ASR model, it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset used to fine-tune the model.

XLS-R was pretrained on audio data of Babel, Multilingual LibriSpeech (MLS), Common Voice, VoxPopuli, and VoxLingua107 at a sampling rate of 16kHz. Common Voice, in its original form, has a sampling rate of 48kHz, thus we will have to downsample the fine-tuning data to 16kHz in the following.

XLS-R was pretrained on audio data of Babel, Multilingual LibriSpeech (MLS), Common Voice, VoxPopuli, and VoxLingua107 at a sampling rate of 16kHz. Common Voice, in its original form, has a sampling rate of 48kHz, thus we will have to downsample the fine-tuning data to 16kHz in the following.

A Wav2Vec2FeatureExtractor object requires the following parameters to be instantiated:

1. Feature_size: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal

2. Sampling_rate: The sampling rate at which the model is trained on.
padding_value: For batched inference, shorter inputs need to be padded with a specific value

3. Do_normalize: Whether the input should be zero-mean-unit-variance normalized or not. Usually, speech models perform better when normalizing the input
return_attention_mask: Whether the model should make use of an attention_mask for batched inference. In general, XLS-R models checkpoints should always use the attention_mask.

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)


In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

## Preprocess Data

In [ ]:
cv_swahili_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/e1af3253b811f6ea12d16411bc452950702d93aec19f951229c67078f572fb63/sw_train_0/common_voice_sw_31361756.mp3',
 'array': array([-6.03961325e-14, -1.35003120e-13, -2.06057393e-13, ...,
        -2.27993539e-07, -2.43553586e-07, -1.32081709e-07]),
 'sampling_rate': 48000}

In [ ]:
from datasets import load_metric, Audio
cv_swahili_train = cv_swahili_train.cast_column("audio", Audio(sampling_rate=16_000))
cv_swahili_validate = cv_swahili_validate.cast_column("audio", Audio(sampling_rate=16_000))
cv_swahili_test = cv_swahili_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
cv_swahili_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/e1af3253b811f6ea12d16411bc452950702d93aec19f951229c67078f572fb63/sw_train_0/common_voice_sw_31361756.mp3',
 'array': array([ 0.00000000e+00, -9.09494702e-12, -1.63709046e-11, ...,
         6.51447039e-08, -9.04973803e-08, -2.16870063e-07]),
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(cv_swahili_train)-1)

print(cv_swahili_train[rand_int]["sentence"])
ipd.Audio(data=cv_swahili_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

juu ya ukweli kwamba zeno alijenga balcony iliyozuia maoni ya anthemius


In [ ]:
rand_int = random.randint(0, len(cv_swahili_train)-1)

print("Target text:", cv_swahili_train[rand_int]["sentence"])
print("Input array shape:", cv_swahili_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", cv_swahili_train[rand_int]["audio"]["sampling_rate"])

Target text: wamejaliwa watoto watatu nyla abbas na mahdi
Input array shape: (102528,)
Sampling rate: 16000


## Preprocess the dataset to the format expected by Wave2Vec2ForCTC leveraging Wav2Vec2Processor

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
cv_swahili_train = cv_swahili_train.map(prepare_dataset, remove_columns=cv_swahili_train.column_names)
cv_swahili_validate = cv_swahili_validate.map(prepare_dataset, remove_columns=cv_swahili_validate.column_names)
cv_Swahili_test = cv_swahili_test.map(prepare_dataset, remove_columns=cv_swahili_test.column_names)

Map:   0%|          | 0/32277 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/10977 [00:00<?, ? examples/s]

Map:   0%|          | 0/10978 [00:00<?, ? examples/s]

In [ ]:
from datasets import concatenate_datasets

# Concatenate cv_swahili_train and cv_swahili_validate
combined_train_validate = concatenate_datasets([cv_swahili_train, cv_swahili_validate])

## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's Trainer for which we essentially need to do the following:

1. Define a data collator. In contrast to most NLP
models, XLS-R has a much larger input length than output length. E.g., a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLS-R requires a special padding data collator, which we will define below

2. Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a compute_metrics function accordingly

3. Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

4. Define the training configuration.

### Define data collator

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

<ipython-input-51-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC, AutoModelForCTC

# model = Wav2Vec2ForCTC.from_pretrained(
#     "facebook/wav2vec2-xls-r-300m",
#     ctc_loss_reduction="mean",
#     pad_token_id=processor.tokenizer.pad_token_id,
#     vocab_size=len(processor.tokenizer),
# )
#model = AutoModelForCTC.from_pretrained("AntonyG/fine-tune-wav2vec2-large-xls-r-1b-sw")
model = AutoModelForCTC.from_pretrained("Joshua-Abok/finetuned_wav2vec_asr")
# model.enable_input_require_grads()

In [ ]:
model.to("cuda")
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1913: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


In [ ]:
# from peft import LoraConfig, get_peft_model, TaskType

# lora_config = LoraConfig(
#     r=16, # Rank
#     lora_alpha=32,
#     target_modules=["q_proj", "v_proj"],
#     lora_dropout=0.05,
#     bias="none",
# )
# model = get_peft_model(model, lora_config)
print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 311258269
all model parameters: 315468445
percentage of trainable model parameters: 98.67%


In [ ]:
from transformers import TrainingArguments, get_linear_schedule_with_warmup

# Check if you are running on a CUDA-enabled device before enabling FP16
import torch
if torch.cuda.is_available():
    fp16_enabled = True
else:
    fp16_enabled = False
    print("CUDA device not available. Disabling FP16.")

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=15,
  gradient_checkpointing=True,
  fp16=fp16_enabled,  # Enable FP16 only if CUDA is available
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
  remove_unused_columns=False
)

# Define the scheduler parameters
num_warmup_steps = int(training_args.max_steps * 0.1)

## Pass all instances to Trainer, ready for training

In [ ]:
from transformers import Trainer
# from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import AdamW

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=combined_train_validate,
    eval_dataset=cv_Swahili_test,
    tokenizer=processor.feature_extractor,
)

# Create the learning rate scheduler
optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)
# scheduler = OneCycleLR(
#     optimizer,
#     total_steps=1000,
#     epochs=training_args.num_train_epochs,
#     steps_per_epoch=len(cv_swahili_train),
#     pct_start=0.3,  # Percentage of the cycle that's a rising learning rate
#     max_lr=4*training_args.learning_rate,    # peak learning rate
#     final_div_factor=25.0,  # in the end LR will decay to initial_LR/final_div_factor
# )
total_steps = 1000
scheduler = CosineAnnealingLR(
    optimizer,
    T_max=total_steps,
    eta_min=0,  # Minimum learning rate
)

# Define a function to update the learning rate during training
def update_lr():
  scheduler.step()

In [ ]:
# Check if this is the correct implementation of the CosineAnnealingLR. It looks like a better learning rate schedulers.
# from torch.optim import AdamW
# from torch.optim.lr_scheduler import CosineAnnealingLR

# optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)
# total_steps = 1000
# epochs = training_args.num_train_epochs
# steps_per_epoch = len(cv_swahili_train)

# scheduler = CosineAnnealingLR(
#     optimizer,
#     T_max=total_steps,
#     eta_min=0,  # Minimum learning rate
# )

 To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the group_tokens=False parameter has to be passed. If we wouldn't pass this parameter a word like "hello" would incorrectly be encoded, and decoded as "helo".
2
2
  The blank token allows the model to predict a word, such as "hello" by forcing it to insert the blank token between the two l's. A CTC-conform prediction of "hello" of our model would be [PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD].

## Training

In [ ]:
# Training loop
for epoch in range(training_args.num_train_epochs):
    trainer.train()
    update_lr()
    trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
400,0.075100,0.271850,0.168000
800,0.121600,0.283317,0.206800
1200,0.138900,0.275259,0.189208
1600,0.136300,0.276866,0.185300
2000,0.132900,0.270723,0.192137
2400,0.132900,0.288053,0.182867
2800,0.131100,0.271615,0.190188
3200,0.127100,0.280063,0.190091
3600,0.130100,0.276147,0.187088
4000,0.126900,0.285467,0.183470


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Buffered data was truncated after reaching the output size limit.

In [ ]:
trainer.push_to_hub()